# Circuit Generation

Construct a memory experiment circuit and inspect the output.


### ErrorModel and CircuitBuildOptions in Circuit Generation

`get_cardinal_circuit(...)` now takes structured objects instead of four separate noise scalars.

- `ErrorModel(idle_error, sqgate_error, tqgate_error, spam_error)` defines the noise channels.
- `CircuitBuildOptions(get_all_detectors, noisy_zeroth_round, noisy_final_meas)` defines circuit-generation behavior.

Tuple support for asymmetric Pauli channels:

- `idle_error` and `sqgate_error`: `(px, py, pz)` for `DEPOLARIZE1`
- `tqgate_error`: 15-tuple ordered as
  `(IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ)` for `DEPOLARIZE2`


In [1]:
import numpy as np
import stim

from quits.circuit import check_overlapping_CX
from quits import ErrorModel, CircuitBuildOptions, get_cardinal_circuit
from quits.decoder import (
    sliding_window_bposd_circuit_mem,
    sliding_window_bposd_phenom_mem,
    sliding_window_bplsd_phenom_mem,
)
from quits.qldpc_code import BpcCode, HgpCode, QlpCode
from quits.simulation import get_stim_mem_result


In [2]:
# Common parameters
seed = 1
p = 1e-3
num_rounds = 10
num_trials = 100
W, F = 5, 3
max_iter, osd_order = 10, 1


In [3]:
# Define structured noise model and circuit-construction options
error_model = ErrorModel(
    idle_error=p,
    sqgate_error=p,
    tqgate_error=p,
    spam_error=p,
)

circuit_build_options = CircuitBuildOptions(
    get_all_detectors=False,
    noisy_zeroth_round=True,
    noisy_final_meas=False,
)


In [4]:
# Build a small HGP code
h = np.loadtxt(
    "../parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt",
    dtype=int,
)
code = HgpCode(h, h)
code.build_circuit(seed=22)
# For this particular parity check matrix, seed=22 is the one that gives entangling depth 8 (other seeds may give entangling depth 12).
report = code.verify_css_logicals()
print("verify_css_logicals", report)


verify_css_logicals {'css_condition': True, 'lz_commutes_with_X': True, 'lx_commutes_with_Z': True, 'rank_hz': 108, 'rank_hx': 108, 'rank_lz': 9, 'rank_lx': 9, 'k_expected': 9, 'lz_independent_mod_Z_stabilizers': True, 'lx_independent_mod_X_stabilizers': True, 'rank_hz_plus_lz': 117, 'rank_hx_plus_lx': 117, 'dim_ker_hz': 117, 'dim_ker_hx': 117, 'hx_plus_lx_spans_ker_hz': True, 'hz_plus_lz_spans_ker_hx': True, 'same_logicals_ZX_anticommute': True, 'different_logicals_ZX_commute': True, 'all_tests_passed': True}


In [5]:
# Generate memory experiment circuit
circuit = stim.Circuit(
    get_cardinal_circuit(code, error_model=error_model, num_rounds=num_rounds, basis="Z", circuit_build_options=circuit_build_options)
)
check_overlapping_CX(circuit, verbose=True);
print(circuit)


No overlapping CX gates found.
R 0 1 2 3 4 5 6 7 8 9 10 11 21 22 23 24 25 26 27 28 29 30 31 32 42 43 44 45 46 47 48 49 50 51 52 53 63 64 65 66 67 68 69 70 71 72 73 74 84 85 86 87 88 89 90 91 92 93 94 95 105 106 107 108 109 110 111 112 113 114 115 116 126 127 128 129 130 131 132 133 134 135 136 137 147 148 149 150 151 152 153 154 155 156 157 158 168 169 170 171 172 173 174 175 176 177 178 179 189 190 191 192 193 194 195 196 197 198 199 200 210 211 212 213 214 215 216 217 218 219 220 221 231 232 233 234 235 236 237 238 239 240 241 242 264 265 266 267 268 269 270 271 272 285 286 287 288 289 290 291 292 293 306 307 308 309 310 311 312 313 314 327 328 329 330 331 332 333 334 335 348 349 350 351 352 353 354 355 356 369 370 371 372 373 374 375 376 377 390 391 392 393 394 395 396 397 398 411 412 413 414 415 416 417 418 419 432 433 434 435 436 437 438 439 440
X_ERROR(0.001) 0 1 2 3 4 5 6 7 8 9 10 11 21 22 23 24 25 26 27 28 29 30 31 32 42 43 44 45 46 47 48 49 50 51 52 53 63 64 65 66 67 68 69 70 